In [0]:
import pandas as pd
from fbprophet import Prophet 
df = pd.read_csv('https://raw.githubusercontent.com/danielcaraway/data/master/Zip_Zhvi_SingleFamilyResidence.csv', encoding='latin')
# df.head()

In [0]:
columns = df.columns[:7].values
region_reference = pd.DataFrame(data=df, columns=columns)
# region_reference.head()

In [0]:
to_drop = ['RegionID',
 'City',
 'State',
 'Metro',
 'CountyName',
 'SizeRank']
just_numbers = df.drop(to_drop, axis=1)

In [0]:
df_t = just_numbers.set_index('RegionName').T

In [0]:
df_t.reset_index(inplace=True)

In [0]:
df_t['year'] = df_t.apply(lambda x: x['index'].split('-')[0], axis=1)

In [45]:
df_t.head()

RegionName,index,10025,60657,10023,77494,60614,77449,10002,77084,79936,60640,11226,10467,78660,94109,10016,37013,32162,60647,11201,11235,11375,90250,78130,37211,10029,10009,77573,60618,77584,10011,20002,10128,28269,78613,77433,78572,30349,79912,75052,...,56030,501,7703,10020,10979,50137,84316,84408,73019,83281,4033,4109,830,86343,2714,2872,99104,89405,40404,82310,82938,544,820,821,822,95721,31421,95375,1063,1097,62059,75599,36872,93282,20052,21759,22649,6230,43738,year
0,1996-04,NaN,355664.0,NaN,197907.0,537402.0,97337.0,NaN,97318.0,82436.0,244560.0,215949.0,233362.0,142708.0,547676.0,NaN,111580.0,114090.0,165184.0,515214.0,228908.0,286114.0,174520.0,NaN,111136.0,NaN,NaN,NaN,212656.0,138762.0,NaN,140355.0,NaN,130993.0,160722.0,163056.0,NaN,95687.0,123875.0,97549.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298904.0,NaN,NaN,70659.0,NaN,142896.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996
1,1996-05,NaN,354736.0,NaN,196854.0,536919.0,97145.0,NaN,97177.0,82406.0,244663.0,215494.0,231848.0,142715.0,547569.0,NaN,112190.0,113879.0,165105.0,510121.0,230004.0,285161.0,174451.0,NaN,111666.0,NaN,NaN,NaN,212797.0,139376.0,NaN,139228.0,NaN,131559.0,160373.0,162894.0,NaN,96288.0,123815.0,97541.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298977.0,NaN,NaN,72780.0,NaN,142141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996
2,1996-06,NaN,355404.0,NaN,195911.0,539044.0,96967.0,NaN,97052.0,82350.0,246601.0,216281.0,230539.0,142832.0,536539.0,NaN,112943.0,112845.0,165876.0,504884.0,229929.0,284662.0,174208.0,NaN,112164.0,NaN,NaN,NaN,212897.0,139277.0,NaN,138072.0,NaN,132346.0,160177.0,163253.0,NaN,97009.0,123574.0,97528.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,299260.0,NaN,NaN,74538.0,NaN,140385.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996
3,1996-07,NaN,355863.0,NaN,197300.0,540137.0,96645.0,NaN,96719.0,82301.0,248032.0,215954.0,230632.0,143012.0,536211.0,NaN,113853.0,111769.0,166797.0,505134.0,229400.0,284759.0,174168.0,NaN,112734.0,NaN,NaN,NaN,212300.0,139221.0,NaN,137244.0,NaN,133091.0,159982.0,163367.0,NaN,97411.0,123431.0,97604.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296621.0,NaN,NaN,75273.0,NaN,139381.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996
4,1996-08,NaN,357309.0,NaN,197320.0,542594.0,96671.0,NaN,96609.0,82242.0,250914.0,216436.0,230275.0,143201.0,536288.0,NaN,114655.0,110567.0,168084.0,514560.0,230332.0,284361.0,174272.0,NaN,113264.0,NaN,NaN,NaN,212398.0,139633.0,NaN,136714.0,NaN,133707.0,160245.0,164164.0,NaN,97823.0,123261.0,97825.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,297102.0,NaN,NaN,75638.0,NaN,138956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996


In [0]:
by_year = pd.DataFrame(df_t.groupby('year').mean())

In [0]:
by_year_t = by_year.reset_index()
by_year_t['year'] = by_year_t['year'].astype('datetime64[ns]') 

In [0]:
by_year_t.set_index('year', inplace=True)


In [0]:
by_year_t2 = by_year_t.T

In [0]:
by_year_t2.to_csv('zips_by_year.csv')

In [0]:
by_year_t2.reset_index(inplace=True)

In [52]:
# by_year_t2['year2'] = by_year_t2['year'].astype('datetime64[ns]') 
by_year_t2

year,RegionName,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,2000-01-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00
0,10025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.524024e+06,1.622857e+06,1.740413e+06,1.880199e+06,1.776032e+06,1.673746e+06,1.426824e+06,1.425603e+06,1.430720e+06,1.369437e+06,1.314699e+06,1.347533e+06,1.354015e+06,1.468749e+06,1.506797e+06,1.445712e+06,1.383775e+06
1,60657,359727.333333,378192.833333,389723.833333,453431.50,521798.916667,586183.833333,628344.916667,6.691198e+05,7.073292e+05,7.562488e+05,8.070008e+05,8.102088e+05,8.231563e+05,7.720613e+05,7.460868e+05,7.097424e+05,7.105657e+05,7.915682e+05,8.466048e+05,8.935743e+05,9.266148e+05,9.397620e+05,9.536783e+05,9.566285e+05
2,10023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.158906e+06,1.177896e+06,1.263354e+06,1.257324e+06,1.286307e+06,1.431668e+06,1.487505e+06,1.625684e+06,1.592752e+06,1.564584e+06,1.498365e+06
3,77494,197676.777778,209368.250000,206195.250000,212468.75,223155.583333,226383.916667,226765.000000,2.311428e+05,2.401042e+05,2.466654e+05,2.498909e+05,2.580278e+05,2.607339e+05,2.610133e+05,2.674436e+05,2.601717e+05,2.648112e+05,2.834968e+05,3.155041e+05,3.351682e+05,3.324876e+05,3.300152e+05,3.334704e+05,3.347144e+05
4,60614,544883.444444,573575.666667,616168.166667,706178.75,804170.666667,905689.416667,983081.333333,1.042087e+06,1.086231e+06,1.164892e+06,1.234957e+06,1.246696e+06,1.179971e+06,1.070229e+06,9.843876e+05,9.093856e+05,8.680966e+05,9.592943e+05,1.032766e+06,1.095943e+06,1.142199e+06,1.175260e+06,1.194322e+06,1.190550e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30429,20052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.944576e+05,8.113819e+05,7.966302e+05,8.477740e+05,9.229986e+05,9.932791e+05,1.023301e+06,1.056330e+06,1.158182e+06,1.272855e+06,1.338094e+06
30430,21759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.792929e+05,2.671424e+05,2.685321e+05,2.635818e+05,2.583762e+05,2.651046e+05,2.834632e+05,2.879907e+05,2.831099e+05,2.543791e+05,2.405582e+05,2.132769e+05
30431,22649,NaN,NaN,NaN,166094.00,174561.416667,181176.333333,187060.583333,1.998489e+05,2.226023e+05,2.559476e+05,2.691606e+05,2.653517e+05,2.515810e+05,2.367340e+05,2.269193e+05,2.060190e+05,1.997838e+05,1.948958e+05,1.905518e+05,1.881821e+05,1.910163e+05,1.957183e+05,2.069102e+05,2.265459e+05
30432,6230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.968482e+05,2.687641e+05,2.522892e+05,2.332192e+05,2.237026e+05,2.360420e+05,2.427438e+05,2.539362e+05,2.449571e+05,2.589315e+05,2.661801e+05,2.646891e+05


In [0]:
mine = by_year_t2[by_year_t2['RegionName'] == 90039]

In [0]:
# mine['2019']

In [0]:
df = by_year_t2.copy()

In [0]:
def get_dream_spreadsheet(row):
  print(row)

  # return { "region": row['RegionName'], 
  #                   "return": 'PROPHET TBD', 
  #                   "prediction_2018": 'PROPHET TBD',
  #                   "actual_2018": row.contains(2018),
  #                   "prediction_2019": 'PROPHET TBD',
  #                   "actual_2019": row['2019']
  #                   }

In [0]:
# df['dream'] = df.apply(lambda x: get_dream_spreadsheet, axis=1)

In [58]:
df

year,RegionName,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,2000-01-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00
0,10025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.524024e+06,1.622857e+06,1.740413e+06,1.880199e+06,1.776032e+06,1.673746e+06,1.426824e+06,1.425603e+06,1.430720e+06,1.369437e+06,1.314699e+06,1.347533e+06,1.354015e+06,1.468749e+06,1.506797e+06,1.445712e+06,1.383775e+06
1,60657,359727.333333,378192.833333,389723.833333,453431.50,521798.916667,586183.833333,628344.916667,6.691198e+05,7.073292e+05,7.562488e+05,8.070008e+05,8.102088e+05,8.231563e+05,7.720613e+05,7.460868e+05,7.097424e+05,7.105657e+05,7.915682e+05,8.466048e+05,8.935743e+05,9.266148e+05,9.397620e+05,9.536783e+05,9.566285e+05
2,10023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.158906e+06,1.177896e+06,1.263354e+06,1.257324e+06,1.286307e+06,1.431668e+06,1.487505e+06,1.625684e+06,1.592752e+06,1.564584e+06,1.498365e+06
3,77494,197676.777778,209368.250000,206195.250000,212468.75,223155.583333,226383.916667,226765.000000,2.311428e+05,2.401042e+05,2.466654e+05,2.498909e+05,2.580278e+05,2.607339e+05,2.610133e+05,2.674436e+05,2.601717e+05,2.648112e+05,2.834968e+05,3.155041e+05,3.351682e+05,3.324876e+05,3.300152e+05,3.334704e+05,3.347144e+05
4,60614,544883.444444,573575.666667,616168.166667,706178.75,804170.666667,905689.416667,983081.333333,1.042087e+06,1.086231e+06,1.164892e+06,1.234957e+06,1.246696e+06,1.179971e+06,1.070229e+06,9.843876e+05,9.093856e+05,8.680966e+05,9.592943e+05,1.032766e+06,1.095943e+06,1.142199e+06,1.175260e+06,1.194322e+06,1.190550e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30429,20052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.944576e+05,8.113819e+05,7.966302e+05,8.477740e+05,9.229986e+05,9.932791e+05,1.023301e+06,1.056330e+06,1.158182e+06,1.272855e+06,1.338094e+06
30430,21759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.792929e+05,2.671424e+05,2.685321e+05,2.635818e+05,2.583762e+05,2.651046e+05,2.834632e+05,2.879907e+05,2.831099e+05,2.543791e+05,2.405582e+05,2.132769e+05
30431,22649,NaN,NaN,NaN,166094.00,174561.416667,181176.333333,187060.583333,1.998489e+05,2.226023e+05,2.559476e+05,2.691606e+05,2.653517e+05,2.515810e+05,2.367340e+05,2.269193e+05,2.060190e+05,1.997838e+05,1.948958e+05,1.905518e+05,1.881821e+05,1.910163e+05,1.957183e+05,2.069102e+05,2.265459e+05
30432,6230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.968482e+05,2.687641e+05,2.522892e+05,2.332192e+05,2.237026e+05,2.360420e+05,2.427438e+05,2.539362e+05,2.449571e+05,2.589315e+05,2.661801e+05,2.646891e+05


In [0]:
test = df.copy()
test_sm = test[:100]
zip_table = []
def get_prophet_predictions(row):
  mini = pd.DataFrame(row)
  mini['ds'] = row.index
  mini['y'] = row.values
  df = mini.iloc[1:]

  m = Prophet()
  m.fit(df)
  invest_price = df.tail(1)
  future = m.make_future_dataframe(periods=60, freq='M')
  fcst = m.predict(future)
  sell_price = fcst.tail(1)

  roi = sell_price['trend'].values[0] - invest_price['y'].values[0]
  return pd.Series((fcst, roi))
  # fig = m.plot(fcst)
  # zip_table.append({ "zipcode": zip_name, "return": roi })  


In [82]:
test_sm[['forecast','roi']] = test_sm.apply(lambda x: get_prophet_predictions(x), axis=1)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 12.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 12.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 18.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run pr

In [0]:
# from datetime import datetime
# datestring = '1996'
# dt = datetime.strptime(datestring, '%Y')

In [67]:
test_sm

year,RegionName,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,2000-01-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00,prophet
0,10025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.524024e+06,1.622857e+06,1.740413e+06,1.880199e+06,1.776032e+06,1.673746e+06,1.426824e+06,1.425603e+06,1.430720e+06,1.369437e+06,1.314699e+06,1.347533e+06,1.354015e+06,1.468749e+06,1.506797e+06,1.445712e+06,1.383775e+06,-1.934741e+06
1,60657,359727.333333,378192.833333,389723.833333,453431.50,521798.916667,586183.833333,628344.916667,6.691198e+05,7.073292e+05,7.562488e+05,8.070008e+05,8.102088e+05,8.231563e+05,7.720613e+05,7.460868e+05,7.097424e+05,7.105657e+05,7.915682e+05,8.466048e+05,8.935743e+05,9.266148e+05,9.397620e+05,9.536783e+05,9.566285e+05,-2.851431e+05
2,10023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.158906e+06,1.177896e+06,1.263354e+06,1.257324e+06,1.286307e+06,1.431668e+06,1.487505e+06,1.625684e+06,1.592752e+06,1.564584e+06,1.498365e+06,-1.210629e+05
3,77494,197676.777778,209368.250000,206195.250000,212468.75,223155.583333,226383.916667,226765.000000,2.311428e+05,2.401042e+05,2.466654e+05,2.498909e+05,2.580278e+05,2.607339e+05,2.610133e+05,2.674436e+05,2.601717e+05,2.648112e+05,2.834968e+05,3.155041e+05,3.351682e+05,3.324876e+05,3.300152e+05,3.334704e+05,3.347144e+05,-1.493322e+05
4,60614,544883.444444,573575.666667,616168.166667,706178.75,804170.666667,905689.416667,983081.333333,1.042087e+06,1.086231e+06,1.164892e+06,1.234957e+06,1.246696e+06,1.179971e+06,1.070229e+06,9.843876e+05,9.093856e+05,8.680966e+05,9.592943e+05,1.032766e+06,1.095943e+06,1.142199e+06,1.175260e+06,1.194322e+06,1.190550e+06,-7.385802e+05


In [0]:
test = df.copy()
test_sm = test[:5]
zip_table = []
def get_ds_y(ds, y):
  print(ds)
  print(y)



In [0]:
# test_sm['prophet'] = test_sm.apply(lambda x: get_ds_y(x, x['RegionName']), axis=1)

In [38]:
test_sm.columns

Index([       'RegionName', 1996-01-01 00:00:00, 1997-01-01 00:00:00,
       1998-01-01 00:00:00, 1999-01-01 00:00:00, 2000-01-01 00:00:00,
       2001-01-01 00:00:00, 2002-01-01 00:00:00, 2003-01-01 00:00:00,
       2004-01-01 00:00:00, 2005-01-01 00:00:00, 2006-01-01 00:00:00,
       2007-01-01 00:00:00, 2008-01-01 00:00:00, 2009-01-01 00:00:00,
       2010-01-01 00:00:00, 2011-01-01 00:00:00, 2012-01-01 00:00:00,
       2013-01-01 00:00:00, 2014-01-01 00:00:00, 2015-01-01 00:00:00,
       2016-01-01 00:00:00, 2017-01-01 00:00:00, 2018-01-01 00:00:00,
       2019-01-01 00:00:00,             'dream'],
      dtype='object', name='year')

In [80]:
test_sm['forecast'][0]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1996-01-01,54797.173399,1.668473e+06,1.984566e+06,54797.173399,54797.173399,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,0.0,0.0,0.0,1.831904e+06
1,1997-01-01,33114.677559,1.648501e+06,1.971987e+06,33114.677559,33114.677559,1.780916e+06,1.780916e+06,1.780916e+06,1.780916e+06,1.780916e+06,1.780916e+06,0.0,0.0,0.0,1.814030e+06
2,1998-01-01,11491.423511,1.648667e+06,1.953012e+06,11491.423511,11491.423511,1.783806e+06,1.783806e+06,1.783806e+06,1.783806e+06,1.783806e+06,1.783806e+06,0.0,0.0,0.0,1.795297e+06
3,1999-01-01,-10131.830537,1.621309e+06,1.945657e+06,-10131.830537,-10131.830537,1.782535e+06,1.782535e+06,1.782535e+06,1.782535e+06,1.782535e+06,1.782535e+06,0.0,0.0,0.0,1.772403e+06
4,2000-01-01,-31755.084585,1.588174e+06,1.897493e+06,-31755.084585,-31755.084585,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,1.777107e+06,0.0,0.0,0.0,1.745352e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2023-08-31,-543740.172555,-7.772837e+05,-4.685001e+05,-543742.773628,-543737.647811,-7.668381e+04,-7.668381e+04,-7.668381e+04,-7.668381e+04,-7.668381e+04,-7.668381e+04,0.0,0.0,0.0,-6.204240e+05
80,2023-09-30,-545517.113333,-1.332335e+06,-1.020407e+06,-545519.759906,-545514.503983,-6.355799e+05,-6.355799e+05,-6.355799e+05,-6.355799e+05,-6.355799e+05,-6.355799e+05,0.0,0.0,0.0,-1.181097e+06
81,2023-10-31,-547353.285470,-1.552874e+06,-1.243726e+06,-547355.978142,-547350.594548,-8.601640e+05,-8.601640e+05,-8.601640e+05,-8.601640e+05,-8.601640e+05,-8.601640e+05,0.0,0.0,0.0,-1.407517e+06
82,2023-11-30,-549130.226247,-4.229907e+05,-1.109694e+05,-549132.971062,-549127.466991,2.815769e+05,2.815769e+05,2.815769e+05,2.815769e+05,2.815769e+05,2.815769e+05,0.0,0.0,0.0,-2.675533e+05


In [0]:
def predictions(df):
  df['forecast'] = 'XXX'
  df['sell_price'] = 'XXX'
  df['roi'] = 'XXX'
  return df

In [0]:
test_sm = test_sm.apply(predictions, axis=1)

In [70]:
test_sm

year,RegionName,1996-01-01 00:00:00,1997-01-01 00:00:00,1998-01-01 00:00:00,1999-01-01 00:00:00,2000-01-01 00:00:00,2001-01-01 00:00:00,2002-01-01 00:00:00,2003-01-01 00:00:00,2004-01-01 00:00:00,2005-01-01 00:00:00,2006-01-01 00:00:00,2007-01-01 00:00:00,2008-01-01 00:00:00,2009-01-01 00:00:00,2010-01-01 00:00:00,2011-01-01 00:00:00,2012-01-01 00:00:00,2013-01-01 00:00:00,2014-01-01 00:00:00,2015-01-01 00:00:00,2016-01-01 00:00:00,2017-01-01 00:00:00,2018-01-01 00:00:00,2019-01-01 00:00:00,prophet,2018,2019,2020
0,10025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.524024e+06,1.622857e+06,1.740413e+06,1.880199e+06,1.776032e+06,1.673746e+06,1.426824e+06,1.425603e+06,1.430720e+06,1.369437e+06,1.314699e+06,1.347533e+06,1.354015e+06,1.468749e+06,1.506797e+06,1.445712e+06,1.383775e+06,-1.934741e+06,XXX,XXX,XXX
1,60657.0,359727.333333,378192.833333,389723.833333,453431.50,521798.916667,586183.833333,628344.916667,6.691198e+05,7.073292e+05,7.562488e+05,8.070008e+05,8.102088e+05,8.231563e+05,7.720613e+05,7.460868e+05,7.097424e+05,7.105657e+05,7.915682e+05,8.466048e+05,8.935743e+05,9.266148e+05,9.397620e+05,9.536783e+05,9.566285e+05,-2.851431e+05,XXX,XXX,XXX
2,10023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.158906e+06,1.177896e+06,1.263354e+06,1.257324e+06,1.286307e+06,1.431668e+06,1.487505e+06,1.625684e+06,1.592752e+06,1.564584e+06,1.498365e+06,-1.210629e+05,XXX,XXX,XXX
3,77494.0,197676.777778,209368.250000,206195.250000,212468.75,223155.583333,226383.916667,226765.000000,2.311428e+05,2.401042e+05,2.466654e+05,2.498909e+05,2.580278e+05,2.607339e+05,2.610133e+05,2.674436e+05,2.601717e+05,2.648112e+05,2.834968e+05,3.155041e+05,3.351682e+05,3.324876e+05,3.300152e+05,3.334704e+05,3.347144e+05,-1.493322e+05,XXX,XXX,XXX
4,60614.0,544883.444444,573575.666667,616168.166667,706178.75,804170.666667,905689.416667,983081.333333,1.042087e+06,1.086231e+06,1.164892e+06,1.234957e+06,1.246696e+06,1.179971e+06,1.070229e+06,9.843876e+05,9.093856e+05,8.680966e+05,9.592943e+05,1.032766e+06,1.095943e+06,1.142199e+06,1.175260e+06,1.194322e+06,1.190550e+06,-7.385802e+05,XXX,XXX,XXX


In [90]:
len(set(df.RegionName))

30434

In [0]:
len_test = df.copy()

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/danielcaraway/data/master/Zip_Zhvi_SingleFamilyResidence.csv', encoding='latin')



In [93]:
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,...,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1486790.0,1490909.0,1493113.0,1500163.0,1497306.0,1496223.0,1498588.0,1504899.0,1511536.0,1509462.0,1518963.0,1518670.0,1526823.0,1512395.0,1503332,1483368,1468482,1461572,1454050,1454111,1451781,1449835,1450723,1440314,1433973,1429073,1428637,1425987,1407384,1391270,1380332,1386270,1394397,1404225,1406599,1399918,1380178,1358401,1350481,1345845
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,355664.0,354736.0,355404.0,355863.0,357309.0,359906.0,362935.0,366493.0,369236.0,371964.0,374364.0,376322.0,377455.0,380637.0,381917.0,384322.0,383195.0,381889.0,378162.0,374642.0,373445.0,370798.0,369891.0,367735.0,369673.0,371771.0,378334.0,386284.0,395034.0,403790.0,412949.0,421560.0,428867.0,...,926166.0,926066.0,929987.0,931155.0,933542.0,935872.0,945765.0,950893.0,950923.0,945971.0,940106.0,937789.0,937761.0,935120.0,931248,932154,941705,953981,959260,960954,959673,959082,954912,950660,948543,948906,951721,954743,957441,961651,966123,967557,965155,960225,956709,953095,950684,948136,946838,945928
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1627167.0,1630578.0,1630922.0,1629389.0,1620963.0,1608791.0,1600559.0,1600247.0,1598869.0,1595751.0,1595421.0,1597321.0,1600114.0,1584614.0,1566777,1543592,1531018,1537936,1551681,1568820,1573613,1578249,1582848,1583553,1580068,1573052,1563902,1550264,1531141,1521399,1520816,1526676,1525174,1516721,1504180,1492086,1480934,1468075,1454870,1438313
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197907.0,196854.0,195911.0,197300.0,197320.0,198443.0,197736.0,198462.0,199158.0,200824.0,201754.0,202558.0,205017.0,207485.0,208972.0,209013.0,211437.0,215407.0,217938.0,217189.0,214825.0,211272.0,209083.0,206815.0,206114.0,205047.0,204507.0,204444.0,203423.0,203923.0,204949.0,207069.0,207697.0,...,329476.0,328831.0,328912.0,329142.0,329312.0,330345.0,331281.0,331534.0,330699.0,330184.0,330013.0,329488.0,329015.0,328902.0,329123,330287,331699,332214,332264,332737,333805,334293,333806,333609,333839,334129,334753,334497,335272,335363,335789,335035,334542,334176,334363,334127,334458,334460,334679,334309
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,537402.0,536919.0,539044.0,540137.0,542594.0,546190.0,550015.0,554436.0,557214.0,560140.0,562389.0,565003.0,566844.0,574208.0,578552.0,583864.0,582314.0,581470.0,578402.0,575431.0,574291.0,575269.0,578602.0,583276.0,588869.0,596347.0,604371.0,615222.0,625944.0,640232.0,651765.0,662952.0,671169.0,...,1141840.0,1140892.0,1144867.0,1147812.0,1153703.0,1159757.0,1174693.0,1185995.0,1191061.0,1187382.0,1179120.0,1174446.0,1174467.0,1173676.0,1172203,1176623,1186741,1197817,1201566,1204015,1204371,1201598,1195307,1189358,1186838,1185950,1187917,1190385,1191138,1193324,1198898,1203423,1204840,1199747,1194591,1188702,1183500,1179024,1175407,1174008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
to_drop = "1996-04	1996-05	1996-06	1996-07	1996-08	1996-09	1996-10	1996-11	1996-12 2019-01	2019-02	2019-03	2019-04	2019-05	2019-06	2019-07	2019-08	2019-09	2019-10	2019-11	2019-12".split()
df_97_18 = df.drop(to_drop, axis=1)


In [99]:
df_97_18

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,...,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1359948.0,1366669.0,1380733.0,1398869.0,1416633.0,1427321.0,1441979.0,1458433.0,1470286.0,1475276.0,1480967.0,1483113.0,1486790.0,1490909.0,1493113.0,1500163.0,1497306.0,1496223.0,1498588.0,1504899.0,1511536.0,1509462.0,1518963.0,1518670.0,1526823.0,1512395.0,1503332,1483368,1468482,1461572,1454050,1454111,1451781,1449835,1450723,1440314,1433973,1429073,1428637,1425987
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,371964.0,374364.0,376322.0,377455.0,380637.0,381917.0,384322.0,383195.0,381889.0,378162.0,374642.0,373445.0,370798.0,369891.0,367735.0,369673.0,371771.0,378334.0,386284.0,395034.0,403790.0,412949.0,421560.0,428867.0,432591.0,435328.0,436075.0,437380.0,439142.0,444487.0,452506.0,460445.0,466996.0,...,902232.0,900170.0,897701.0,900732.0,907374.0,921322.0,930106.0,932259.0,929455.0,927464.0,929221.0,928802.0,926166.0,926066.0,929987.0,931155.0,933542.0,935872.0,945765.0,950893.0,950923.0,945971.0,940106.0,937789.0,937761.0,935120.0,931248,932154,941705,953981,959260,960954,959673,959082,954912,950660,948543,948906,951721,954743
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1488991.0,1489134.0,1498052.0,1518543.0,1556960.0,1598874.0,1627446.0,1646238.0,1648779.0,1646252.0,1635584.0,1630020.0,1627167.0,1630578.0,1630922.0,1629389.0,1620963.0,1608791.0,1600559.0,1600247.0,1598869.0,1595751.0,1595421.0,1597321.0,1600114.0,1584614.0,1566777,1543592,1531018,1537936,1551681,1568820,1573613,1578249,1582848,1583553,1580068,1573052,1563902,1550264
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,200824.0,201754.0,202558.0,205017.0,207485.0,208972.0,209013.0,211437.0,215407.0,217938.0,217189.0,214825.0,211272.0,209083.0,206815.0,206114.0,205047.0,204507.0,204444.0,203423.0,203923.0,204949.0,207069.0,207697.0,207549.0,207531.0,208211.0,209158.0,210300.0,211477.0,213057.0,214438.0,215233.0,...,337110.0,337166.0,336857.0,337243.0,336604.0,335648.0,334774.0,334665.0,335081.0,333881.0,332303.0,330534.0,329476.0,328831.0,328912.0,329142.0,329312.0,330345.0,331281.0,331534.0,330699.0,330184.0,330013.0,329488.0,329015.0,328902.0,329123,330287,331699,332214,332264,332737,333805,334293,333806,333609,333839,334129,334753,334497
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,560140.0,562389.0,565003.0,566844.0,574208.0,578552.0,583864.0,582314.0,581470.0,578402.0,575431.0,574291.0,575269.0,578602.0,583276.0,588869.0,596347.0,604371.0,615222.0,625944.0,640232.0,651765.0,662952.0,671169.0,676136.0,679232.0,681187.0,686043.0,689523.0,697088.0,705580.0,715431.0,723215.0,...,1107490.0,1105375.0,1102707.0,1108665.0,1114867.0,1131654.0,1142148.0,1150378.0,1149627.0,1148597.0,1147756.0,1145955.0,1141840.0,1140892.0,1144867.0,1147812.0,1153703.0,1159757.0,1174693.0,1185995.0,1191061.0,1187382.0,1179120.0,1174446.0,1174467.0,1173676.0,1172203,1176623,1186741,1197817,1201566,1204015,1204371,1201598,1195307,1189358,1186838,1185950,1187917,1190385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
df_nona = df_97_18.dropna()

In [101]:
len(df_nona)

12655